<a href="https://colab.research.google.com/github/the-Luke-R/Bot_Concurseiro/blob/main/Bot_concurseiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install -q -U google-generativeai
!pip install -q -U PyPDF2

In [ ]:
"""Fazemos o upload do arquivo sobre o qual serão feitas as perguntas

Recomendamos este arquivo (marco legal dos jogos eletrônicos) para teste:
https://legis.senado.leg.br/sdleg-getter/documento?dm=9206085&ts=1715006513087&disposition=inline"""

from google.colab import files
upload = files.upload()
user_file = list(upload.keys())[0]

In [28]:
from pathlib import Path
import hashlib
import google.generativeai as genai
import PyPDF2

genai.configure(api_key="SUA_CHAVE_GOOGLE_AI_STUDIO")

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]

  with open(pathname, 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)

    for page_num in range(num_pages):
      page = pdf_reader.pages[page_num]
      page_content = page.extract_text()
      parts.append(f"--- PAGE {page_num + 1} ---")
      parts.append(page_content)

  return parts


convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages(user_file)
  },
  {
    "role": "user",
    "parts": ["Você e um bot especialista em criar perguntas para estudantes ou pessoas que tem interesse em aprender. Com base no arquivo fornecido crie 5 perguntas com 5 alternativas cada utilizando o seguinte formato:\n\nPERGUNTAS\n\n1. Texto da pergunta\nA ( ) texto da alternativa\nB ( ) texto da alternativa\nC ( ) texto da alternativa\nD ( ) texto da alternativa\nE ( ) texto da alternativa\n\n2. Texto da pergunta\nA ( ) texto da alternativa\nB ( ) texto da alternativa\nC ( ) texto da alternativa\nD ( ) texto da alternativa\nE ( ) texto da alternativa\n...\n\nApós o usuário responder você deve dizer as respostas e quantas perguntas o usuário acertou seguido da porcentagem\n\nEXEMPLO:\n\n-- RESPOSTAS --\n1 - E\n2 - C\n...\n\nVocê acertou 2/5 perguntas (40%)\n"]
  },
  {
    "role": "model",
    "parts": ["## PERGUNTAS\n\n1. Qual é o propósito principal da Convenção sobre os Direitos das Pessoas com Deficiência?\nA ( ) Estabelecer um sistema de cotas para pessoas com deficiência em empregos públicos.\nB ( ) Criar um tribunal internacional para julgar crimes contra pessoas com deficiência.\nC ( ) Promover a igualdade de gênero entre homens e mulheres com deficiência.\nD ( ) Assegurar a educação inclusiva para crianças com deficiência em todos os níveis.\nE ( ) Promover, proteger e assegurar o exercício pleno e equitativo de todos os direitos humanos e liberdades fundamentais por todas as pessoas com deficiência. \n\n2.  Segundo a Constituição da República Federativa do Brasil, em qual situação a pena de morte é permitida?\nA ( ) Em casos de crime hediondo.\nB ( ) Em casos de latrocínio.\nC ( ) Em casos de tráfico de drogas.\nD ( ) Em casos de guerra declarada.\nE ( ) A pena de morte é proibida em qualquer situação.\n\n3.  De acordo com a Constituição Brasileira, qual o órgão responsável por processar e julgar o Presidente da República em caso de crimes de responsabilidade?\nA ( ) Supremo Tribunal Federal.\nB ( ) Superior Tribunal de Justiça.\nC ( ) Conselho Nacional de Justiça.\nD ( ) Senado Federal.\nE ( ) Câmara dos Deputados.\n\n4.  Qual a porcentagem mínima da receita resultante de impostos que a União deve aplicar anualmente na manutenção e desenvolvimento do ensino?\nA ( ) 10% \nB ( ) 15%\nC ( ) 18%\nD ( ) 20%\nE ( ) 25%\n\n5.  A Constituição Brasileira define a família como a base da sociedade e garante a ela especial proteção do Estado. Quais formas de união são reconhecidas como entidade familiar pela Constituição?\nA ( )  Apenas o casamento civil entre um homem e uma mulher.\nB ( )  Casamento civil e união estável entre homem e mulher.\nC ( )  Casamento civil, união estável entre homem e mulher, e a comunidade formada por qualquer dos pais e seus descendentes.\nD ( )  Qualquer união entre duas pessoas, independentemente de seu gênero ou parentesco.\nE ( )  A Constituição não define as formas de união reconhecidas como entidade familiar. \n\n---\n\nResponda às perguntas com a letra da alternativa correspondente, como no exemplo a seguir:\n\n1 - A, 2 - B, 3 - C, 4 - D, 5 - E"]
  },
  {
    "role": "user",
    "parts": ["1 - E\n2 - D\n3 - B\n4 - A\n5 - B"]
  },
  {
    "role": "model",
    "parts": ["-- RESPOSTAS --\n1 - E\n2 - D\n3 - D\n4 - C\n5 - C\n\nVocê acertou 3/5 perguntas (60%)"]
  },
])


In [ ]:
create_question_prompt = f"Com base no arquivo {user_file}, crie 5 perguntas com 5 alternativas cada no final \n---------------------------------------------------------------------------------"
print(create_question_prompt)
convo.send_message(create_question_prompt)
print(convo.last.text)
user_answers = input("Digites as suas respostas:\n\n")
convo.send_message(user_answers)
print(convo.last.text)

prompt = input("Você gostaria de responder mais perguntas? \n\n digite 's' para continuar e 'n' para encerrar e aperte enter \n\n")
while prompt == "s":
  convo.send_message(create_question_prompt)
  print(convo.last.text)
  user_answers = input("Digites as suas respostas:\n\n")
  convo.send_message(user_answers)
  print(convo.last.text)
  prompt = input("Você gostaria de responder mais perguntas? \n\n digite 's' para continuar e 'n' para encerrar e aperte enter \n\n")